<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/09_Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP(Naturala Language Processing) and NLU (Natural Language Understanding) :

`A handful of example natural language processing (NLP) and natural language understanding (NLU) problems. These are also often referred to as sequence problems (going from one sequence to another)`

NLP has the goal of deriving information out of natural language (Could be sequences text or speech). Another common term of NLP probelems is sequence to sequence problems (seq2seq)

**Let's see where do we use it : Which are reffered to as a Sequence Problems**

1. MultiLabel Classification
2. Machine Translation
3. Text Generation
4. Voice Assistant

**Types of Sequence problems :(input to output types)**
1. One to one  (one word translation)
2. One to many (Image captioning, possible to have multiple captions)
3. Many to one (Sentiment analysis maybe a youtube comment, BitCoin Price Prediction)
4. many to many (Part-of-speech tagging in natural language processing. Each word in the input sentence may correspond to multiple possible part-of-speech tags.)
5. many to many (DNA sequence alignment. Aligning two DNA sequences where each base in one sequence may correspond to multiple bases in the other.)







# What will we do in this section :         

* Downloading and preparing the text data
* How to prepare text data for modeling (Tokenization and embedding)
* Setting up multiple modeling experiments with Recurrent Neural Networks (RNN)
* Building a text feature extraction model using TensorFlow Hub
* Finding the most wrong prediction examples
* Using a model we've built to make predictions on the text from the wild



**NLP Inputs and outputs :**

`Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)`

esource: For a great overview of NLP and the different problems within it, read the article A Simple Introduction to [Natural Language Processing](https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32).



# STEPS  WE FOLLOW IN CREATING IN RNN / CNN FOR NLP

1. Get the data ready (Turn all data into numbers as neural networks can't handle text or natural language and make sure all of your tensors are in right shape pad sequence which don't fit )
2. Build or pick a pretrained model (to suit your problem)
3. Fit the model to the data and make a prediction
4. Evaluate the model
5. Improve through Experimentation
6. Save and reload your trained model



# TYPICAL ARCHITECTURE OF A RECCURENT NEURAL NETWORK

**What is RNN :**  is class of Artificial Neural Networks where connections between nodes from a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behaviour.

1. Input Text
2. Input Layers
3. Text vectorization layer
4. Embedding (Turns mapping of text vectors to embedding matrix (representing how words relate Ex King-man+woman = queen))
5. RNN Cells (Finds Pattern in sequence) `Ex : Simple RNN, LSTM, GRU`
6. Hidden Activations : Adds non linearity to learned features usually use `tanh`
7. Pooling layer : Reduces the dimensionality of the learned features usually Conv1D models `Ex : GlobalAveragePooling1D or GlobalMaxPool1D`
8. Fully Connected layer : Further refines learned features from recurrent layers (Dense layer)
9. Output layer (Takes learned feature and outputs them in shape of target labels) Ex : `output_shape = [number_of_classes] (Disaster or not)`
10. Output Activation : Adds non-linearities to output layer.






# Check GPU

In [ ]:
!nvidia-smi

Wed Jan 10 20:13:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get helper functions

In [ ]:
# Get helper functions Import the series of the helper functions using the below link

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Import the functions using import
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-10 20:13:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-01-10 20:13:15 (12.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Get the Text Dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as diaster or not diaster)

See the original source here :https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip



In [ ]:
#Get the data

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-01-10 20:13:19--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.251.8.207, 142.251.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   985KB/s    in 0.6s    

2024-01-10 20:13:20 (985 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Becoming one with the data and visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use python: https://realpython.com/python-csv/#:~:text=Reading%20from%20a%20CSV%20file,which%20does%20the%20heavy%20lifting.


 but I prefer to get visual straight way

 So another way to do this to use pandas.



In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [ ]:
#Shuffle training dataframe
train_df_shuffled = train_df.sample(frac =1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
#What does the test dataframe look like

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
#How many total samples
len(train_df) + len(test_df)

10876

In [ ]:
#Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create a random indexes not higher than the total num
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target : {target}", "(real diaster)" if target >0 else "(not diaster)")
  print(f"Text :  {text}\n")
  print("---\n")

Target : 0 (not diaster)
Text :  Conning Builds Strong Case for Portfolio #Diversification for Property-Casualty Insurers http://t.co/33FbR25t1O

---

Target : 0 (not diaster)
Text :  Building Muscle With Feast And Famine Feeding Most diet plans are based around losing weight but what should youÛ_ https://t.co/lUe3waeGpI

---

Target : 1 (real diaster)
Text :  #news #crimes Police ID victim in fatal crash that closed I-65 near Franklin: One person is d... http://t.co/9h0ym9OFsv #justice #courts

---

Target : 1 (real diaster)
Text :  @Bilsko and suddenly I'm inundated with research. @humofthecity

---

Target : 0 (not diaster)
Text :  I liked a @YouTube video http://t.co/M9YdA5k6jf Spyro 3 Texture Hacks - 'Desolate Gardens' [In-game]

---



# Split data into training and validation sets



In [ ]:
from sklearn.model_selection import train_test_split



In [ ]:
# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, # use 10% of training data for validation
                                                                            random_state = 42)

In [ ]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
#Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with the text problems, one of the first things you'll have to do before you can build a model is to convert text to numbers.

`There are few ways to do this, namely :`  
**Tokenization :** Straight mapping from token to number (Can be modelled but quickly get's too big)

**Ebedding :** Richer representation of relationships between tokens (Can limit size + can be learned) : Create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeding can be learned)





# Text Vectorization (Tokenization)

In [ ]:
train_sentences[:5]


array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
# Text Vectorization (Tokenization)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=10000, # How many words in the vocabulary (automatically add  <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams = None, # Create groups of n-words
                                    output_mode = "int", # how to map tokens to numbers
                                    output_sequence_length = None, #how long do you want your sequences to be
                                    pad_to_max_tokens = True)

In [ ]:
len(train_sentences[0].split())

7

In [ ]:
#Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
#Setup text vectorization variables
max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [ ]:
#Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)


In [ ]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [ ]:
#Create a sample sentence and tokenize it
sample_sentance = "There is a flood in my street"
text_vectorizer([sample_sentance])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from a training dataset and tokenize it

random_sentence = random.choice(train_sentences)
print(f"Original text : {random_sentence}\
        \n\nVectorized version :")
text_vectorizer([random_sentence])


Original text : @FAIRx818x @PlayOverwatch @BlizzardCS please blizzard we love you        

Vectorized version :


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 9732,    1,  170,  863,   46,  110,   12,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
#Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get the all of the unique words in the
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocabulary : {len(words_in_vocab)}")
print(f"Top 5 words in vocabulary : {top_5_words}")
print(f"Bottom 5 words in vocabulary : {bottom_5_words}")

Number of words in vocabulary : 10000
Top 5 words in vocabulary : ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 words in vocabulary : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating an Embedding using an Embedding Layer

Turns a positive integers (indexes) into dense vectors of fixed size

To make our embeddding,  we're going to use tensorflow layer and for more infor please visit the below link : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about  for our embedding layer :      

* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented 100 long.
* `input_length`  = Length of the sequences being passes to the embedding layer



In [ ]:
from tensorflow.keras import layers

#Create a embedding layer
embedding = layers.Embedding(input_dim = max_vocab_length, # Set input shape
                             output_dim = 128, # Output shape
                             input_length = max_length) # How long is each input
embedding

In [ ]:
#Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text : {random_sentence}\
      \n \n Embedded Version :" )

#Embed the random sentences (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text : Gov. Brown links CA wildfire to drought http://t.co/jEvrCWUdpm      
 
 Embedded Version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00603534,  0.01727798,  0.02360897, ...,  0.04863263,
          0.03953264, -0.01640649],
        [-0.03649829, -0.04334352, -0.04591824, ..., -0.03708433,
          0.00961936,  0.03675929],
        [-0.01965528,  0.00908601,  0.00993595, ..., -0.03153352,
         -0.01276609,  0.02497189],
        ...,
        [-0.03117989, -0.03362696, -0.03755151, ...,  0.02223945,
         -0.00067176,  0.01587787],
        [-0.03117989, -0.03362696, -0.03755151, ...,  0.02223945,
         -0.00067176,  0.01587787],
        [-0.03117989, -0.03362696, -0.03755151, ...,  0.02223945,
         -0.00067176,  0.01587787]]], dtype=float32)>

In [ ]:
#Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00603534,  0.01727798,  0.02360897, -0.01660442, -0.04751561,
        -0.01874633, -0.03149616, -0.00023615, -0.01613492,  0.00685573,
         0.0289796 , -0.01763568, -0.00863833, -0.01143676, -0.04912416,
        -0.01816938, -0.02867154,  0.04183456, -0.02214744,  0.01293642,
        -0.03601097, -0.02892971,  0.0465121 ,  0.03254482,  0.01868517,
        -0.01661606,  0.02608942,  0.03183576, -0.01949058, -0.03809036,
        -0.02684317, -0.02105392,  0.00853001,  0.0095997 ,  0.02870841,
         0.04300935, -0.00753913,  0.0086407 , -0.03597816,  0.0403272 ,
         0.0324312 , -0.0233555 , -0.01274375, -0.02379854,  0.00259112,
        -0.04096467,  0.04369416,  0.02272684,  0.00620432, -0.0031466 ,
        -0.01461   , -0.00713986,  0.01700724,  0.03027941, -0.04742549,
        -0.01384715, -0.01292821,  0.01721484,  0.02779669,  0.04131   ,
        -0.03664132,  0.00393475,  0.04732256, -0.03584962, -0.01563818,
  

## Modeling a text dataset (Running a series of experiments)

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

`Model 0`: Naive Bayes  TF-IDF (baseline) [Sklearn ML Map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

`Model 1`: Feed-forward neural network (dense model)

`Model 2`: LSTM model

`Model 3`: GRU model

`Model 4`: Bidirectional-LSTM model

`Model 5`: 1D Convolutional Neural Network

`Model 6`: TensorFlow Hub Pretrained Feature Extractor (Using transfer learning for NLP)

`Model 7`: Same as model 6 with 10% of training data


Model 0 is the simplest to acquire a baseline which we'll expect each other of the other deeper models to beat.

**Each experiment will go through the following steps:**

*   Construct the model
*   Train the model
*   Make predictions with the model
*   Track prediction evaluation metrics for later comparison

      
      
      
      

# Model 0 : Getting a baseline

As with all machine learning modelling experiments, it's importat to create baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

**Note :** It's common practice to use non-DL algorithms as baseline because of their speed and then later using DL to see if you can improve upon them.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modeling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # Convert words to numbers using tfidf
    ('clf', MultinomialNB()) # model the text
])

#Fit the pipeline to training the data
model_0.fit(train_sentences, train_labels)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our Baseline model achieves an accuracy : {round(baseline_score * 100)}%")

Our Baseline model achieves an accuracy : 79%


In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# Creating an evaluation function for our experiments

We could evaluate all of our model predictions with different metrics every time however, this would be cumbersome and could

Let's create 1 to compare our model's predictions with the truth labels using the following metrics :     

* Accuracy
* Precision
* Recall
* F-1 Score

For a deep overview of many different evaluation methods, see the sklearn documentation :     https://scikit-learn.org/stable/modules/model_evaluation.html


In [ ]:
# Function to evaluate : Accuracy, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred) :
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall, and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy ": model_accuracy,
                   "precision ": model_precision,
                   "recall ": model_recall,
                   "f1 ": model_f1}

  return model_results

In [ ]:
#Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

# Model 1: A Simple Dense Model (Feed-forward neural network)

In [ ]:
# Create a tensorboard callback (Need to create a new one for each model)
from helper_functions import create_tensorboard_callback

#Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"




In [ ]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
# x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [ ]:
# Compile model
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
# Fit the model
model_1_history = model_1.fit(x = train_sentences,
                              y=train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240110-201324
Epoch 1/5
215/215 [==============================] - 9s 31ms/step - loss: 0.6103 - accuracy: 0.6955 - val_loss: 0.5355 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4405 - accuracy: 0.8231 - val_loss: 0.4809 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3462 - accuracy: 0.8599 - val_loss: 0.4594 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2839 - accuracy: 0.8914 - val_loss: 0.4616 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2376 - accuracy: 0.9102 - val_loss: 0.4779 - val_accuracy: 0.7808


In [ ]:
#Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4779 - accuracy: 0.7808


[0.4779190123081207, 0.7808399200439453]

In [ ]:
#Make some predictions
model_1_pre_probs = model_1.predict(val_sentences)
model_1_pre_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [ ]:
embedding.weights

[<tf.Variable 'embedding/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.04668647, -0.02043865, -0.03357662, ...,  0.0081479 ,
          0.01303764,  0.00162928],
        [ 0.04048793, -0.02270064, -0.01979429, ..., -0.05265208,
          0.04131158, -0.00824692],
        [ 0.02905487,  0.0255617 ,  0.04033086, ..., -0.04275548,
         -0.02913261, -0.05470839],
        ...,
        [-0.04180991, -0.04076862, -0.01398172, ...,  0.04396531,
         -0.00767739, -0.00710384],
        [-0.04809929,  0.05375234,  0.03403461, ..., -0.05464147,
          0.07350039,  0.00096093],
        [-0.10422913,  0.11286591,  0.08530624, ..., -0.11127651,
          0.08378681, -0.06354219]], dtype=float32)>]

In [ ]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.34499177],
       [0.7720961 ],
       [0.9977906 ],
       [0.13844378],
       [0.10329475],
       [0.94258887],
       [0.90738195],
       [0.9929143 ],
       [0.9677292 ],
       [0.32062933]], dtype=float32)

In [ ]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [ ]:
#COnvert model prediction probabilities to label formats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [ ]:
#Calculate our model_1 results
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results


{'accuracy ': 78.08398950131233,
 'precision ': 0.7844894281510498,
 'recall ': 0.7808398950131233,
 'f1 ': 0.77812284684276}

In [ ]:
baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

In [ ]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings

In [ ]:
#Get the vocabulary from the text vectorizatiion layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
# Model 1 summary

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [ ]:
# Get the weight matrix of embeding layer
# These are the neumarical representations of each token in our training data, which have been learned for 5 epochs

embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights
print(embed_weights.shape) # Same size as vocab size and ebedding_dim (Output dim of our embedding layer)

(10000, 128)


Now we have got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, Tensorflow has a handy tool called projector : https://projector.tensorflow.org/

And Tensorflow also has an incredible guide on word embeddings themselves :
https://www.tensorflow.org/text/guide/word_embeddings


In [ ]:
embed_weights

array([[-0.04668647, -0.02043865, -0.03357662, ...,  0.0081479 ,
         0.01303764,  0.00162928],
       [ 0.04048793, -0.02270064, -0.01979429, ..., -0.05265208,
         0.04131158, -0.00824692],
       [ 0.02905487,  0.0255617 ,  0.04033086, ..., -0.04275548,
        -0.02913261, -0.05470839],
       ...,
       [-0.04180991, -0.04076862, -0.01398172, ...,  0.04396531,
        -0.00767739, -0.00710384],
       [-0.04809929,  0.05375234,  0.03403461, ..., -0.05464147,
         0.07350039,  0.00096093],
       [-0.10422913,  0.11286591,  0.08530624, ..., -0.11127651,
         0.08378681, -0.06354219]], dtype=float32)

In [ ]:
#Create embedding files (We got this from tensorFlow's word embeddings documentation)

"""import io

out_v = io.open('vectors.tsv', 'w', encoding = 'utf-8')
out_m = io.open('metadata.tsv', 'w', encoding = 'utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # Skip 0, it's padding
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()"""

'import io\n\nout_v = io.open(\'vectors.tsv\', \'w\', encoding = \'utf-8\')\nout_m = io.open(\'metadata.tsv\', \'w\', encoding = \'utf-8\')\n\nfor index, word in enumerate(words_in_vocab):\n  if index == 0:\n    continue # Skip 0, it\'s padding\n  vec = embed_weights[index]\n  out_v.write(\'\t\'.join([str(x) for x in vec]) + "\n")\n  out_m.write(word + "\n")\nout_v.close()\nout_m.close()'

Reseourses : Word2Vec https://jalammar.github.io/illustrated-word2vec/


In [ ]:
#Download files from Colab to upload to projector
"""
try :
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass"""

"\ntry :\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception as e:\n  pass"

Once you've downloaded the embedding vectors and metadata, you can visualize them using Embedding Vector tool:

Go to http://projector.tensorflow.org/
Click on "Load data"
Upload the two files you downloaded (embedding_vectors.tsv and embedding_metadata.tsv)
Explore
Optional: You can share the data you've created by clicking "Publish"
What do you find?

Are words with similar meanings close together?

Remember, they might not be. The embeddings we downloaded are how our model interprets words, not necessarily how we interpret them.

Also, since the embedding has been learned purely from Tweets, it may contain some strange values as Tweets are a very unique style of natural language.

🤔 Question: Do you have to visualize embeddings every time?

No. Although helpful for gaining an intuition of what natural language embeddings are, it's not completely necessary. Especially as the dimensions of your vocabulary and embeddings grow, trying to comprehend them would become an increasingly difficult task.

# Recurrent Neural Networks (RNN's) :     

RNN's are useful for sequence of data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

**If you want an overview of the internals of a recurrent neural network, see the following :**     
  * MIT Sequence modeling lecture : https://www.youtube.com/watch?v=ySEx_Bqxvvo
  * Chris Olah's intro to LSTMs : https://colah.github.io/posts/2015-08-Understanding-LSTMs/
  * Andre Karpathy's the unreasonable effectiveness of recurrent neural networks : https://karpathy.github.io/2015/05/21/rnn-effectiveness/





# Model. 2: LSTM [Long Short Term Memory] :

## One of the most popular LSTM Cells :

Our structure of an RNN typically looks like this:      



```
Input(text) -> Tokenize -> Embedding -> Layers(RNNs/Debse) -> Output(Label probability)

```







In [ ]:
#Create an LSTM Model

from tensorflow.keras.layers import LSTM
inputs = layers.Input(shape=(1,), dtype ="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units = 64, return_sequences=True)(x) # It would be better if it's multiple of 8, when you are stacking RNN cells together u need to return sequences = True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [ ]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Compile the model
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])



In [ ]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240110-201340
Epoch 1/5
215/215 [==============================] - 10s 35ms/step - loss: 0.2200 - accuracy: 0.9226 - val_loss: 0.5198 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1570 - accuracy: 0.9416 - val_loss: 0.6483 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1290 - accuracy: 0.9536 - val_loss: 0.6884 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1058 - accuracy: 0.9594 - val_loss: 0.8720 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0857 - accuracy: 0.9647 - val_loss: 0.8855 - val_accuracy: 0.7822


In [ ]:
#Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


24/24 [==============================] - 0s 2ms/step


array([[1.0935001e-02],
       [8.1073594e-01],
       [9.9982649e-01],
       [2.0427056e-01],
       [7.4281252e-04],
       [9.9730206e-01],
       [6.5244746e-01],
       [9.9988806e-01],
       [9.9978954e-01],
       [7.1543801e-01]], dtype=float32)

In [ ]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
#Calculate Model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy ': 78.21522309711287,
 'precision ': 0.7833242410577732,
 'recall ': 0.7821522309711286,
 'f1 ': 0.7805300645444495}

In [ ]:
baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

# Model 3 : GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

the GRU cell has similar features ato an LSTM cell but has less parameters.

Resource : https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be

In [ ]:
#Build an RNN using the GRU cell

from tensorflow.keras.layers import GRU
inputs = layers.Input(shape=(1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.GRU(64, return_sequences =True)(x) # If you want to stack recurrent layers on top of each other you need return_sequences = True
# print(x.shape)
# x = layers.LSTM(42, return_sequences =True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation ="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [ ]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [ ]:
# Compile the model
model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data =(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240110-201358
Epoch 1/5
215/215 [==============================] - 9s 34ms/step - loss: 0.1604 - accuracy: 0.9361 - val_loss: 0.7225 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0807 - accuracy: 0.9686 - val_loss: 0.8004 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0680 - accuracy: 0.9731 - val_loss: 1.1731 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0596 - accuracy: 0.9747 - val_loss: 1.1904 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0521 - accuracy: 0.9762 - val_loss: 1.2391 - val_accuracy: 0.7730


In [ ]:
#Make some predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.8794005e-03],
       [8.9521950e-01],
       [9.9993753e-01],
       [5.8184031e-02],
       [5.8479513e-05],
       [9.9976164e-01],
       [8.3405954e-01],
       [9.9997854e-01],
       [9.9994278e-01],
       [8.7402517e-01]], dtype=float32)

In [ ]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred = model_3_preds)
model_3_results

{'accuracy ': 77.29658792650919,
 'precision ': 0.7753400570774207,
 'recall ': 0.7729658792650919,
 'f1 ': 0.7705764257839071}

#Model 4 :  Bidirectional RNN Model

Resource : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional

Normal RNN's go from left to right (just like you'd read an English sentence) however, a birectional RNN goes from right to left as well as left to right.




In [ ]:
#Build Birectional RNN in tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")


In [ ]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [ ]:
#Compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240110-201415
Epoch 1/5
215/215 [==============================] - 11s 36ms/step - loss: 0.1079 - accuracy: 0.9685 - val_loss: 0.9299 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0503 - accuracy: 0.9777 - val_loss: 1.1803 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0449 - accuracy: 0.9780 - val_loss: 1.2600 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0398 - accuracy: 0.9815 - val_loss: 1.4868 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0447 - accuracy: 0.9787 - val_loss: 1.3015 - val_accuracy: 0.7703


In [ ]:
#Make predictions with our model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[2.3429610e-02],
       [8.0809617e-01],
       [9.9996877e-01],
       [2.0944819e-01],
       [1.7365857e-05],
       [9.9939954e-01],
       [9.5628190e-01],
       [9.9999082e-01],
       [9.9997950e-01],
       [9.9785656e-01]], dtype=float32)

In [ ]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
#Convert pred probs to pred labels
model_4_results = calculate_results(y_true = val_labels,
                                  y_pred = model_4_preds)
model_4_results

{'accuracy ': 77.03412073490814,
 'precision ': 0.7701208199177533,
 'recall ': 0.7703412073490814,
 'f1 ': 0.7695203715374298}

In [ ]:
model_3_results

{'accuracy ': 77.29658792650919,
 'precision ': 0.7753400570774207,
 'recall ': 0.7729658792650919,
 'f1 ': 0.7705764257839071}

# Convolutional Neural Networks for Text (and other types of sequences)

We've used CNNs for images but images are typically 2D (height * width)... however out text is 1D

previouslu we've Conv2D for our image data but now we're going to use Conv1D for text.

The Trypical structure of a Conv1D model for sequence (in our case, text):
I

```
# The Trypical structure of a Conv1D model for sequence (in our case, text):
Inputs(Text) -> Tokanization -> Embedding -> Layer(s) -> (Typically Conv1D + Pooling) -> output (class probabilities)
```

# Model 5 : Conv 1D

For different explanations of parameters see : https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

* CNN explainer for 2D and 1D : https://poloclub.github.io/cnn-explainer/


In [ ]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # TUrns target sequence into embedding
conv_1d = layers.Conv1D(filters = 32, # Hidden units
                        kernel_size = 5,  # this is also reffered to ngram og 5 (meaning it looks at 5 words at a time)
                        strides =1,  # Default
                        activation ="relu",
                        padding ="valid") # Default = "valid", the output is smaller than input shape, "same" means output is same
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [ ]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-3.7348352e-02,  1.7570479e-02, -2.9360399e-02, ...,
          1.1375804e-02,  1.3763272e-02, -1.9686537e-02],
        [ 5.8823233e-03, -1.6000578e-02,  4.9961032e-03, ...,
          4.4675894e-02,  2.6435511e-02,  6.8383440e-02],
        [ 3.7950475e-02, -6.5883607e-02, -2.8510321e-02, ...,
         -3.9376091e-02, -1.2073336e-02,  1.1792532e-03],
        ...,
        [-4.2012978e-02, -2.4593309e-03, -2.7050080e-02, ...,
          8.9407971e-05,  2.7105933e-02,  6.8059103e-03],
        [-4.2012978e-02, -2.4593309e-03, -2.7050080e-02, ...,
          8.9407971e-05,  2.7105933e-02,  6.8059103e-03],
        [-4.2012978e-02, -2.4593309e-03, -2.7050080e-02, ...,
          8.9407971e-05,  2.7105933e-02,  6.8059103e-03]]], dtype=float32)>

In [ ]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.02190487, 0.05618649, 0.02377265, 0.01177888,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.00417053, 0.00193006, 0.03986777, 0.03127235, 0.05043627,
         0.        , 0.        , 0.0664196 , 0.02012935, 0.        ,
         0.02249109, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.01593657, 0.00466411, 0.        , 0.        ,
         0.03366368, 0.        ],
        [0.0455167 , 0.01226519, 0.03551839, 0.03163373, 0.        ,
         0.        , 0.01801725, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.01724469, 0.        , 0.        ,
         0.00689526, 0.        , 0.        , 0.        , 0.01445096,
         0.0494131 , 0.04212502, 0.        , 0.        , 0.02058805,
         0.04981883, 0.        , 0.02013677, 0.        , 0.01185451,
         0.03365489, 0.0200937 ],
        [0.03091259, 0.        , 0.01520435, 0.0090

In [ ]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.04570292, 0.03200354, 0.05618649, 0.03163373, 0.01177888,
        0.01917155, 0.01801725, 0.00478924, 0.05638904, 0.        ,
        0.01869787, 0.03873485, 0.03986777, 0.03127235, 0.05264097,
        0.04038707, 0.01164664, 0.0664196 , 0.02012935, 0.01445096,
        0.0494131 , 0.07252099, 0.        , 0.00559523, 0.02058805,
        0.04981883, 0.01593657, 0.04735672, 0.02079664, 0.01185451,
        0.04235437, 0.02753957]], dtype=float32)>

In [ ]:
#Create a 1-Dimensional convolutional layer to model sequences

from tensorflow.keras import layers
inputs = layers.Input(shape=(1, ), dtype = tf.string)

x= text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation ="relu", padding ="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name = "model_5_Conv1D")

#Compile Conv1D
model_5.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

# get the summary of our Conv1D model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 1)              

In [ ]:
#Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240110-201436
Epoch 1/5
215/215 [==============================] - 9s 33ms/step - loss: 0.1267 - accuracy: 0.9569 - val_loss: 0.9156 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0740 - accuracy: 0.9736 - val_loss: 1.0405 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0618 - accuracy: 0.9759 - val_loss: 1.1400 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0539 - accuracy: 0.9785 - val_loss: 1.1938 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0522 - accuracy: 0.9781 - val_loss: 1.2867 - val_accuracy: 0.7598


In [ ]:
#Make some predictions with our Conv1D model
model_5_pred_probs= model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[8.9301243e-02],
       [6.2480545e-01],
       [9.9975973e-01],
       [7.4101381e-02],
       [2.3374971e-08],
       [9.8745990e-01],
       [9.4870770e-01],
       [9.9992907e-01],
       [9.9999809e-01],
       [8.8449842e-01]], dtype=float32)

In [ ]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
# Evaluate the model 5 predictions
model_5_results = calculate_results(y_true= val_labels,
                                    y_pred = model_5_preds)
model_5_results

{'accuracy ': 75.98425196850394,
 'precision ': 0.7627061506667501,
 'recall ': 0.7598425196850394,
 'f1 ': 0.7568651555223058}

# Model 6 : TensorFLow Hub Pretrained Sentence Encoder

Now we've built a few of our own models. let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder : https://tfhub.dev/google/universal-sentence-encoder/4

See how the USE was in the conference paper created here : https://arxiv.org/abs/1803.11175

In [ ]:
sample_sentance

'There is a flood in my street'

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embed_samples = embed([sample_sentance, "When you can the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01602836  0.01068853  0.02425464 -0.01405769  0.01434428  0.08292626
  0.0196337   0.0616014  -0.00352701 -0.01216415  0.00978647 -0.01248499
  0.01232348  0.09748451  0.06141112 -0.03728352  0.01860887 -0.04669852
  0.00413913 -0.06363907 -0.02469897  0.02713692  0.02284444 -0.00210027
 -0.00630592 -0.03964959  0.02220407  0.00115075 -0.03132181  0.00119528
 -0.0401255   0.04561896 -0.01530597 -0.00175914  0.02173133 -0.08450425
  0.03340027  0.04604555 -0.0248025  -0.08681665  0.00702696 -0.00770477
 -0.01434536  0.07814164 -0.10676058 -0.05152997 -0.00858159 -0.03232231
 -0.03871096  0.02581471], shape=(50,), dtype=float32)


In [ ]:
#Create a keras layer using USE pretrained layer from the the TF Hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # no matter how long the sentence is, the output will be always a feature vector of 512
                                        dtype= tf.string,
                                        trainable = False,
                                        name = "USE")

In [ ]:
#Create a model using sequential API

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation ="sigmoid", name = "output_layer")
], name = "model_6_USE")

# Compile
model_6.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256871809 (979.89 MB)
Trainable params: 73985 (289.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [ ]:
# train a classifier on top of USE pretrained embeddings

model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20240110-201537
Epoch 1/5
215/215 [==============================] - 6s 14ms/step - loss: 0.4727 - accuracy: 0.7856 - val_loss: 0.4399 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3948 - accuracy: 0.8259 - val_loss: 0.4238 - val_accuracy: 0.8202
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3692 - accuracy: 0.8412 - val_loss: 0.4124 - val_accuracy: 0.8320
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3382 - accuracy: 0.8584 - val_loss: 0.4269 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2977 - accuracy: 0.8787 - val_loss: 0.4437 - val_accuracy: 0.8255


In [ ]:
# Make predictions with USE TF hub Model

model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 7ms/step


array([[0.0702542 ],
       [0.9230253 ],
       [0.99931836],
       [0.1663667 ],
       [0.45434558],
       [0.9170299 ],
       [0.99621093],
       [0.99579144],
       [0.98383904],
       [0.0734234 ]], dtype=float32)

In [ ]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(y_true = val_labels,
                                    y_pred = model_6_preds)
model_6_results

{'accuracy ': 82.54593175853019,
 'precision ': 0.8286895266074388,
 'recall ': 0.8254593175853019,
 'f1 ': 0.8238269415090449}

In [ ]:
baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

# Model 7 : TF Hub Pretrained USE but with 10% of training data

Tranfer leraning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data.



In [ ]:
# making data splits like below leads to data leakage(model 7 trained on 10% of the data, outperforms model_6 trained on 100% data)
# Do not make the data splits which leak data from validation/test sets into training set

# # Create subset's of 10% of the training data
# train_10_percent = train_df_shuffled[["text", 'target']].sample(frac = 0.1, random_state = 42)
# #train_10_percent.head(), len(train_10_percent)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()
# len(train_sentences_10_percent), len(train_labels_10_percent)



(761, 761)

**Note :** Be very careful when crating training/val/test splits that you don't leak data across the datasets, otherwise your model evaluation metrics will be wrong. If something looks too good to be true (a model trained on 10% of the data outperforming the same model trained on 100% of data) trust your gut and go back through to find where the erro may lie.

In [ ]:
# Making a better datasent split : No data leakage
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_labels_10_percent)

685

In [ ]:
# Check the number of each label in the updated training data subset

pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [ ]:
# Check the number of targets in our subset of data
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

# Clone Model

To recreate the model same as previous model you have built, You can use the tf.keras.models.clone_models() method, see more https://www.tensorflow.org/api_docs/python/tf/keras/models/clone_model

In [ ]:
# Let's build the model same as model 6

# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation ="sigmoid", name = "output_layer")
], name = "model_7_USE")

#Compile the model
model_7.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

#Get a summary (will be same as model_6)
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 128)               65664     
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256871809 (979.89 MB)
Trainable params: 73985 (289.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [ ]:
#Fit the model to the 10% of the training the data

model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "tf_hub_sentence_encoder_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent_correct_split/20240110-223034
Epoch 1/5
22/22 [==============================] - 3s 47ms/step - loss: 0.6690 - accuracy: 0.5766 - val_loss: 0.6421 - val_accuracy: 0.5840
Epoch 2/5
22/22 [==============================] - 0s 21ms/step - loss: 0.5598 - accuracy: 0.7635 - val_loss: 0.5480 - val_accuracy: 0.7664
Epoch 3/5
22/22 [==============================] - 0s 21ms/step - loss: 0.4338 - accuracy: 0.8394 - val_loss: 0.4880 - val_accuracy: 0.7795
Epoch 4/5
22/22 [==============================] - 0s 21ms/step - loss: 0.3502 - accuracy: 0.8657 - val_loss: 0.5087 - val_accuracy: 0.7808
Epoch 5/5
22/22 [==============================] - 0s 22ms/step - loss: 0.2959 - accuracy: 0.8832 - val_loss: 0.5389 - val_accuracy: 0.7743


In [ ]:
# Make predictions
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]


24/24 [==============================] - 1s 8ms/step


array([[0.03024901],
       [0.3202229 ],
       [0.9746454 ],
       [0.39995408],
       [0.6054829 ],
       [0.3752766 ],
       [0.94987   ],
       [0.8993016 ],
       [0.9488446 ],
       [0.08900744]], dtype=float32)

In [ ]:
# Turn pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 1., 0., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
#Evaluate model 7 predictions
model_7_results = calculate_results(y_true = val_labels,
                                    y_pred = model_7_preds)
model_7_results

{'accuracy ': 77.42782152230971,
 'precision ': 0.7782373722272058,
 'recall ': 0.7742782152230971,
 'f1 ': 0.7712566971963791}

In [ ]:
model_6_results

{'accuracy ': 82.54593175853019,
 'precision ': 0.8286895266074388,
 'recall ': 0.8254593175853019,
 'f1 ': 0.8238269415090449}

# Comparing the performance of all models that we built so far





In [ ]:
#Combine model results into a dataframe
all_model_results = pd.DataFrame({"0_baseline":baseline_results,
                                  "1_simple_dense": model_1_results,
                                  "2_lstm":model_2_results,
                                  "3_gru":model_3_results,
                                  "4_bidirectional":model_4_results,
                                  "5_conv1d":model_5_results,
                                  "6_tf_hub_use_encoder":model_6_results,
                                  "7_tf_hub_use_encoder_10_percent": model_7_results})

all_model_results = all_model_results.transpose()
all_model_results

,accuracy,precision,recall,f1
0_baseline,79.265092,0.811139,0.792651,0.786219
1_simple_dense,78.083990,0.784489,0.780840,0.778123
2_lstm,78.215223,0.783324,0.782152,0.780530
3_gru,77.296588,0.775340,0.772966,0.770576
4_bidirectional,77.034121,0.770121,0.770341,0.769520
5_conv1d,75.984252,0.762706,0.759843,0.756865
6_tf_hub_use_encoder,82.545932,0.828690,0.825459,0.823827
7_tf_hub_use_encoder_10_percent,77.427822,0.778237,0.774278,0.771257


,accuracy,precision,recall,f1
0_baseline,79.265092,0.811139,0.792651,0.786219
1_simple_dense,78.083990,0.784489,0.780840,0.778123
2_lstm,78.215223,0.783324,0.782152,0.780530
3_gru,77.296588,0.775340,0.772966,0.770576
4_bidirectional,77.034121,0.770121,0.770341,0.769520
5_conv1d,75.984252,0.762706,0.759843,0.756865
6_tf_hub_use_encoder,82.545932,0.828690,0.825459,0.823827
7_tf_hub_use_encoder_10_percent,77.427822,0.778237,0.774278,0.771257
